###   Importing necessary library

In [2]:
import os

import tensorflow as tf
import tensorflow as gfile

from typing import List
import numpy as np
from PIL import Image
import time

In [3]:
import cv2

### Implementation of Layers

In [6]:
padding = ['SAME','VALID']

In [9]:
def load(data_path, session, ignore_missing=False):
        """Load network weights.
        data_path: The path to the numpy-serialized network weights
        session: The current TensorFlow session
        ignore_missing: If true, serialized weights for missing layers are ignored.
        """
        data_dict = np.load(data_path, encoding='latin1').item()  # pylint: disable=no-member

        for op_name in data_dict:
            with tf.variable_scope(op_name, reuse=True):
                for param_name, data in iteritems(data_dict[op_name]):
                    try:
                        var = tf.get_variable(param_name)
                        session.run(var.assign(data))
                    except ValueError:
                        if not ignore_missing:
                            raise

In [12]:
def conv(inpt, k_h, k_w, c_o, s_h, s_w, name, relu=True, padding=padding[0], group=1, biased=True):
        # Convolution for a given input and kernel
        with tf.variable_scope(name) as scope:
            kernel = tf.get_variable('weights', shape=[k_h, k_w, c_i // group, c_o], trainable=True)
            output = tf.nn.conv2d(inpt, kernal, [1, s_h, s_w, 1], padding=padding)
            # Add the biases
            if biased:
                biases = tf.get_variable('biases', [c_o])
                output = tf.nn.bias_add(output, biases)
            if relu:
                # ReLU non-linearity
                output = tf.nn.relu(output, name=scope.name)
            return output

In [13]:
def prelu(inpt, name):
        with tf.variable_scope(name):
            i = int(inpt.get_shape()[-1])
            alpha = tf.get_variable('alpha', shape=(i,))
            output = tf.nn.relu(inpt) + tf.multiply(alpha, -tf.nn.relu(-inpt))
        return output

In [14]:
def max_pool(inpt, k_h, k_w, s_h, s_w, name, padding=padding[0]):
        self.validate_padding(padding)
        return tf.nn.max_pool(inpt, ksize=[1, k_h, k_w, 1],strides=[1, s_h, s_w, 1], padding=padding, name=name)

In [20]:
def fully_connected(inpt, num_out, name, relu=True):
        with tf.variable_scope(name):
            input_shape = inpt.get_shape()
            if input_shape.ndims == 4:
                # since the input is spatial.we have to Vectorize it first.
                dim = 1
                for d in input_shape[1:].as_list():
                    dim *= int(d)
                feed_in = tf.reshape(inp, [-1, dim])
            else:
                feed_in, dim = (inpt, input_shape[-1].value)
            weights = tf.get_variable('weights', shape=[dim, num_out])
            biases = tf.get_variable('biases', [num_out])
            op = tf.nn.relu_layer if relu else tf.nn.xw_plus_b
            fc = op(feed_in, weights, biases, name=name)
            return fc

In [19]:
def softmax(target, axis, name=None):
        max_axis = tf.reduce_max(target, axis, keepdims=True)
        target_exp = tf.exp(target - max_axis)
        normalize = tf.reduce_sum(target_exp, axis, keepdims=True)
        softmax = tf.div(target_exp, normalize, name)
        return softmax

## Implementation of MTCNN

### P-Net ( Proposal - Network )

In [ ]:
def pNet(data):
    value = conv(data, 3, 3, 10, 1, 1, padding='VALID', relu=False, name='conv1')
    value = prelu(value, name='PReLU1')
    value = max_pool(value, 2, 2, 2, 2, name='pool1')
    value = conv(value ,3, 3, 16, 1, 1, padding='VALID', relu=False, name='conv2')
    value = prelu(value ,name='PReLU2')
    value = conv(value ,3, 3, 32, 1, 1, padding='VALID', relu=False, name='conv3')
    value = prelu(value ,name='PReLU3')
    val_conv4_1 = conv(value ,1, 1, 2, 1, 1, relu=False, name='conv4-1')
    softmax(3, name='prob1')

    (self.feed('PReLU3')  # pylint: disable=no-value-for-parameter
    .conv(value ,1, 1, 4, 1, 1, relu=False, name='conv4-2'))